<h1>Stock Data Web Scraping using Python</h1>

##### Extracting and Visualizing
I will extract some stock data using Webscraping and API, then display this data in a graph.
To answer this question :
#### What is the impact of profitability and external factors such as collective investor actions on stock prices?

<h2>Table of Contents</h2>
<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ul>
        <li>Use yfinance API to Extract Tesla Stock Data</li>
        <li>Use yfinance API to Extract GME Stock Data</li>
        <li>Use Webscraping to Extract Tesla Revenue Data</li>
        <li>Use Webscraping to Extract GME Revenue Data</li>
        <li>Plot Tesla Stock Graph</li>
        <li>Plot GameStop Stock Graph</li>
        <li>Insights</li>
    </ul>
</div>

<hr>


In [ ]:
!pip install yfinance
!pip install requests
!pip install beautifulsoup4

In [1]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.io as pio
pio.renderers.default = 'notebook'

## Define Graphing Function


In [12]:
# It takes a dataframe with stock data (dataframe must contain Date and Close columns),
# a dataframe with revenue data (dataframe must contain Date and Revenue columns), and the name of the stock.

def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing=0.3)
    stock_data_specific = stock_data[stock_data.Date <= '2021-06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False, height=900, title=stock, xaxis_rangeslider_visible=True)
    fig.show()
    if stock == 'Tesla' :
        fig.write_html('Tesla.html')
    elif stock == 'GameStop' :
        fig.write_html('GameStop.html')

## Use yfinance to Extract Stock Data


In [3]:
def get_stock_data(symbol):
    stock = yf.Ticker(symbol)
    data = stock.history(period="max")
    data.reset_index(inplace=True)
    return data

## Use Webscraping to Extract Tesla Revenue Data


In [4]:
def get_tesla_revenue():
    url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'class': 'historical_data_table'})
    rows = table.tbody.find_all('tr')

    revenue_data = []
    for row in rows:
        cols = row.find_all('td')
        if len(cols) == 2:
            date = cols[0].text
            revenue = cols[1].text.replace('$', '').replace(',', '')
            revenue_data.append((date, revenue))

    # Create DataFrame with columns Date and Revenue
    df = pd.DataFrame(revenue_data, columns=['Date', 'Revenue'])
    return df

## Use Webscraping to Extract GME Revenue Data


In [5]:
def get_gme_revenue():
    url = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'class': 'historical_data_table'})
    rows = table.tbody.find_all('tr')

    revenue_data = []
    for row in rows:
        cols = row.find_all('td')
        if len(cols) == 2:
            date = cols[0].text
            revenue = cols[1].text.replace('$', '').replace(',', '')
            revenue_data.append((date, revenue))

    # Create DataFrame with columns Date and Revenue
    df = pd.DataFrame(revenue_data, columns=['Date', 'Revenue'])
    return df

### Call functions to get Stock and Revenue data of Tesla and GME

In [6]:
# get stock data of Tesla
tesla_data = get_stock_data('TSLA')
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


In [7]:
# get stock data of Game Stop
gme_data = get_stock_data('GME')
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620129,1.693350,1.603296,1.691667,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712708,1.716074,1.670627,1.683251,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658002,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662210,1.603296,1.662210,6892800,0.0,0.0


In [8]:
# get Revenue data of Tesla
tesla_revenue_data = get_tesla_revenue()
tesla_revenue_data.head()

,Date,Revenue
0,2022,81462
1,2021,53823
2,2020,31536
3,2019,24578
4,2018,21461


In [9]:
# get Revenue data of Game Stop
gme_revenue_data = get_gme_revenue()
gme_revenue_data.head()

,Date,Revenue
0,2023,5927
1,2022,6011
2,2021,5090
3,2020,6466
4,2019,8285


## Plot Tesla Stock Graph


Note the graph will only show data upto June 2021.

In [ ]:
pip install nbformat --upgrade

In [13]:
make_graph(tesla_data, tesla_revenue_data, "Tesla")

##### You can view the gragh in `Tesla.html` file

## Plot GameStop Stock Graph


Note the graph will only show data upto June 2021.

In [14]:
make_graph(gme_data, gme_revenue_data, 'GameStop')

##### You can view the gragh in `GameStop.html` file

## Insights

 * The Tesla stock graph revealed a positive correlation between increasing profitability and stock price growth, supporting the notion that improving financial performance drives investor confidence and market valuation.
 * Conversely, the GameStop graph depicted the unexpected surge in stock price triggered by the collective actions of individual investors. This visual representation illustrated the unique dynamics of market sentiment and the potential impact of non-fundamental factors on stock prices.
 * Tesla's profit margins are much higher than GameStop's.
 * Tesla is a more efficient company than GameStop.
 * The overall market conditions and the company's financial performance can also affect the price of a stock.

The graphs I created served as a valuable tool for hedge fund managers, enabling them to compare stock prices with profitability trends. It allowed them to assess the potential risks and opportunities associated with investing in these companies. The data-driven insights facilitated informed decision-making and a deeper understanding of market dynamics.